In [10]:
import cv2

In [14]:
net = cv2.dnn.readNetFromONNX("best (1).onnx")

In [15]:
%cd ../
%cd weights

c:\Users\perez\OneDrive\Documents\GitHub\Yolo
c:\Users\perez\OneDrive\Documents\GitHub\Yolo\weights


In [16]:
classes = [] # Gets our Object names
with open("obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [17]:
import numpy as np
import time

In [18]:
layer_names = net.getLayerNames()
output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]

In [22]:
cap=cv2.VideoCapture(0)

In [23]:
while True:
    # Read a frame from the webcam
    ret, frame = cap.read()
    if not ret:
        break  # Break the loop if no frame is captured
    
    # Prepare the input image for inference (resize, normalize, etc.)
    blob = cv2.dnn.blobFromImage(frame, scalefactor=1.0, size=(300, 300), mean=(127.5, 127.5, 127.5), swapRB=True)
    
    # Set the input blob for the network
    net.setInput(blob)
    
    # Forward pass to get the output layer predictions
    detections = net.forward()
    
    # Process the detections
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:  # Filter out low-confidence detections
            class_id = int(detections[0, 0, i, 1])
            class_name = classes[class_id]
            box = detections[0, 0, i, 3:7] * [frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]]
            (x, y, w, h) = box.astype("int")
            
            # Draw bounding box and label on the frame
            cv2.rectangle(frame, (x, y), (w, h), (0, 255, 0), 2)
            text = f"{class_name}: {confidence:.2f}"
            cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Display the frame with detections
    cv2.imshow("Object Detection", frame)
    
    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\layers\concat_layer.cpp:109: error: (-201:Incorrect size of input array) Inconsistent shape for ConcatLayer in function 'cv::dnn::ConcatLayerImpl::getMemoryShapes'


In [24]:
cap.release()
cv2.destroyAllWindows()